# FlyThruGateAvitary curriculum checks

Goals:
- Create an env with `use_curriculum=True` and inspect observation/action spaces.
- Reset and verify the spawn point per curriculum level.
- Run a few random steps to ensure the env runs.
- Compare actual spawn vs expected position for each level (optional GUI preview).


In [1]:
from gym_pybullet_drones.envs.FlyThruGateAvitary import FlyThruGateAvitary
from gym_pybullet_drones.utils.enums import ObservationType, ActionType
import numpy as np


/home/tuan/Desktop/drone_rl_control/gym_pybullet_drones/envs/BaseAviary.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
pybullet build time: Sep 29 2025 13:26:22


In [2]:
# Create env with curriculum enabled
env = FlyThruGateAvitary(
    obs=ObservationType('kin'),
    act=ActionType('rpm'),
    gui=False,
    record=False,
    use_curriculum=True,
    curriculum_level=0,
    max_curriculum_level=5,
)

print('Observation space:', env.observation_space)
print('Action space:', env.action_space)
print(f'Curriculum level: {env.curriculum_level}/{env.max_curriculum_level}')


[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 3.160000e-10, km 7.940000e-12,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
Observation space: Box(-inf, inf, (19,), float32)
Action space: Box(-1.0, 1.0, (1, 4), float32)
Curriculum level: 0/5


/home/tuan/Desktop/drone_rl_control/venv/lib/python3.12/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/tuan/Desktop/drone_rl_control/venv/lib/python3.12/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [3]:
# Reset once to inspect spawn point and info
obs, info = env.reset()
obs_arr = np.array(obs)
print('Initial obs shape:', obs_arr.shape)
print('Start position (xyz):', obs_arr[:3])
print('Info keys:', list(info.keys()))


Initial obs shape: (19,)
Start position (xyz): [-0.00408028 -0.9         0.24726602]
Info keys: ['position', 'quaternion', 'rpy', 'lin_vel', 'ang_vel', 'time_passed_gate']


In [4]:
# Run a few random steps as a quick smoke test
for step in range(5):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    obs_arr = np.array(obs)
    print(f'Step {step}: reward={reward:.3f}, terminated={terminated}, truncated={truncated}')
    print('  pos:', obs_arr[:3], 'vel:', obs_arr[3:6], 'rpy:', obs_arr[6:9])


Step 0: reward=0.100, terminated=False, truncated=False
  pos: [-0.00408047 -0.8999992   0.24704252] vel: [-1.9650899e-05  8.3753876e-05 -1.1917617e-02] rpy: [-0.00136734 -0.00032602 -0.00641222]
Step 1: reward=0.106, terminated=False, truncated=False
  pos: [-0.00408137 -0.8999874   0.24659277] vel: [ 6.0307888e-05  5.8454863e-04 -1.4716559e-02] rpy: [-0.00085801  0.00287611 -0.02795126]
Step 2: reward=0.112, terminated=False, truncated=False
  pos: [-0.00404643 -0.8999701   0.24619971] vel: [ 0.0025675   0.00011059 -0.00952519] rpy: [ 0.00756181  0.02354486 -0.05530144]
Step 3: reward=0.118, terminated=False, truncated=False
  pos: [-0.00379729 -0.9000337   0.24549939] vel: [ 0.01274577 -0.00425636 -0.02999788] rpy: [ 0.02264929  0.0586581  -0.08618106]
Step 4: reward=0.124, terminated=False, truncated=False
  pos: [-0.00301077 -0.9003531   0.24412332] vel: [ 0.03377333 -0.01457518 -0.0502031 ] rpy: [ 0.03332968  0.10227447 -0.12038951]


In [5]:
def preview_spawn_levels(max_level=5, num_steps=20, use_gui=False, sleep_sec=0.05):
    """Print spawn position for each level and compare to the expected offset.

    If GUI is enabled, each level opens a PyBullet window and runs zero action for a few steps.
    """
    gate_center = np.array([0.0, -1.0, 0.5])  # per reset(): gate + [0,0,0.5]

    for lvl in range(max_level + 1):
        sim_env = FlyThruGateAvitary(
            obs=ObservationType('kin'),
            act=ActionType('rpm'),
            gui=use_gui,
            record=False,
            use_curriculum=True,
            curriculum_level=lvl,
            max_curriculum_level=max_level,
        )

        obs, _ = sim_env.reset()
        obs_arr = np.array(obs)

        expected = gate_center + np.array([0.0, 0.1 + 0.15 * lvl, 0.0])
        delta = obs_arr[:3] - expected
        print(f'Level {lvl}: expected={expected}, start xyz={obs_arr[:3]}, delta={delta}')

        zero_action = np.zeros_like(sim_env.action_space.sample())
        for _ in range(num_steps):
            obs, reward, terminated, truncated, info = sim_env.step(zero_action)
            if use_gui:
                import time
                time.sleep(sleep_sec)
            if terminated or truncated:
                break
        sim_env.close()


In [ ]:
# Quick preview of all levels without GUI
preview_spawn_levels(max_level=env.max_curriculum_level, num_steps=5, use_gui=False)


[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 3.160000e-10, km 7.940000e-12,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
Level 0: expected=[ 0.  -0.9  0.5], start xyz=[ 0.00449689 -0.9         0.252773  ], delta=[ 4.49688546e-03  2.38418579e-08 -2.47227013e-01]
[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 3.160000e-10, km 7.940000e-12,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
Level 1: expected=[ 0

: 

## Visualize spawn with GUI
Uncomment the cell below to open PyBullet for each level (window closes after `num_steps`).


In [ ]:
preview_spawn_levels(max_level=env.max_curriculum_level, num_steps=40, use_gui=True, sleep_sec=0.05)


[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 3.160000e-10, km 7.940000e-12,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
viewMatrixstartThreads creating 1 threads.
starting thread 0
 (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
projectionMatrix (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VEND

In [ ]:
env.close()
